<h3> DATA ANALYSIS - CPD Beats</h3>

1) Specify the Parameters

Beats of District 3: 

311
312
313
314
321
322
323
324
331
332
333
334

In [1]:
district=3  #Chicago police district
delta=.3  #distance threshold in kilometers
tau=30  #time threshold in minutes
beat_array = [311,312,313,314,321,322,323,324,331,332,333,334]

2) Import Libraries

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

3) Import the Chicago shotspotter data and filter for the specified district and beat.

In [3]:
ss_raw = pd.read_csv("Shotspotter_2022.csv")
shotspotter_raw=ss_raw[["BEAT","DATE","DISTRICT","LATITUDE","LONGITUDE"]]
shotspotter_raw=shotspotter_raw[shotspotter_raw["DISTRICT"]==district]
#shotspotter_raw=shotspotter_raw[shotspotter_raw["BEAT"]==beat]
shotspotter_raw=shotspotter_raw.dropna()
for i in shotspotter_raw.index:
    x=shotspotter_raw.loc[i,"DATE"]
    x1=x.split(' ')
    x2=x1[0].split('/')
    shotspotter_raw.loc[i,"time"]=x1[1]
    shotspotter_raw.loc[i,'MONTH'] =int(x2[0])
    shotspotter_raw.loc[i,"day"]=x2[1]
    shotspotter_raw.loc[i,"YEAR"]=int(x2[2])
shotspotter_raw=shotspotter_raw[["DATE","BEAT", "YEAR","MONTH","LATITUDE","LONGITUDE"]]
shotspotter_raw=shotspotter_raw.reset_index(drop=True)
shotspotter_raw.head(1)

,DATE,BEAT,YEAR,MONTH,LATITUDE,LONGITUDE
0,11/29/2019 11:57:40 PM,323.0,2019.0,11.0,41.764712,-87.611835


4) Streamline the columns and created a new column 'spotted' to show which are linked to shotspotter events.

In [4]:
for i in shotspotter_raw.index:
    x=shotspotter_raw.loc[i,"DATE"]
    x1=x.split(' ')
    x2=x1[0].split('/')
    shotspotter_raw.loc[i,'MONTH'] =int(x2[0])
    shotspotter_raw.loc[i,'YEAR'] =int(x2[2])
shotspotter_raw=shotspotter_raw[["DATE","BEAT","YEAR","MONTH","LATITUDE","LONGITUDE"]]
shotspotter_raw.head(2)

,DATE,BEAT,YEAR,MONTH,LATITUDE,LONGITUDE
0,11/29/2019 11:57:40 PM,323.0,2019.0,11.0,41.764712,-87.611835
1,02/06/2018 04:12:47 PM,324.0,2018.0,2.0,41.759659,-87.592333


5) Read in the shooting event data and add a column for spotted (linked) shootings.

In [5]:
shootings_raw = pd.read_csv('Homicide_data_to_2022.csv')
shootings_raw=shootings_raw[shootings_raw["DISTRICT"]==district]
#shootings_raw=shootings_raw[shootings_raw["BEAT"]==beat]
shootings_raw=shootings_raw[shootings_raw["GUNSHOT_INJURY_I"]=='YES']
for i in shootings_raw.index:
    x=shootings_raw.loc[i,"DATE"]
    x1=x.split(' ')
    x2=x1[0].split('/')
    shootings_raw.loc[i,'MONTH'] =int(x2[0])
    shootings_raw.loc[i,'YEAR'] =int(x2[2])
shootings_raw= shootings_raw[['DATE','YEAR','MONTH','DISTRICT','BEAT','LATITUDE', 'LONGITUDE','LOCATION']]
shootings_raw['spotted'] = 0
shootings_raw=shootings_raw.reset_index(drop=True)
shootings_raw.head(2)

,DATE,YEAR,MONTH,DISTRICT,BEAT,LATITUDE,LONGITUDE,LOCATION,spotted
0,01/23/1996 04:35:00 PM,1996.0,1,3.0,313.0,41.784330,-87.613289,POINT (-87.613288547471 41.784329668947),0
1,02/04/1996 07:00:00 AM,1996.0,2,3.0,323.0,41.765432,-87.614685,POINT (-87.61468534612 41.765431654142),0


In [6]:
shootings_raw["BEAT"].value_counts(dropna=False)

321.0    371
322.0    370
324.0    359
312.0    346
323.0    342
313.0    318
331.0    294
332.0    288
334.0    199
311.0    194
314.0    116
333.0    111
Name: BEAT, dtype: int64

6) Import libraries used to link positions and times of shotspotter and shooting incidents.

In [7]:
!!pip install geopy
from geopy import distance
import datetime
import re

7) Define a function to get the linking distribution for a specified year and month

In [8]:
def LD(shooting_data,shotspotter_data,year,month,delta,tau):
    #install libraries
    from geopy import distance
    import datetime
    #get the threshold values
    delta = delta
    tau = tau
    shotspotter_data = shotspotter_data[shotspotter_data["YEAR"]==year]
    shotspotter_data = shotspotter_data[shotspotter_data["MONTH"]==month]
    shooting_data = shooting_data[shooting_data["YEAR"]==year]
    shooting_data = shooting_data[shooting_data["MONTH"]==month]
    shotspotter_data = shotspotter_data.reset_index(drop = True)
    shooting_data = shooting_data.reset_index(drop=True)
    #define the function to determine linkage of shotspotter and shooting incidents based on the threshold values
    def linked(P, T, delta, tau):
        for k in range(P.shape[0]):
            for q in range(P.shape[1]):
                if P[k][q] < delta and T[k][q] < tau and T[k][q] >= 0:
                    shooting_data.loc[k, 'spotted'] = shooting_data.loc[k,'spotted']+ 1
    #Create the P and T matrices
    P = np.zeros((shooting_data.shape[0], shotspotter_data.shape[0]))
    T = np.zeros((shooting_data.shape[0], shotspotter_data.shape[0]))
    for i in shooting_data.index:
        for j in shotspotter_data.index:
            pt1 = [shooting_data.loc[i,"LATITUDE"],shooting_data.loc[i,"LONGITUDE"]]
            pt2 = [shotspotter_data.loc[j,"LATITUDE"],shotspotter_data.loc[j,"LONGITUDE"]]
            P[i][j] = distance.distance(pt1, pt2).km
            strdate1 = shooting_data.loc[i,"DATE"]
            strdate2 = shotspotter_data.loc[j,"DATE"]
            date1 = datetime.datetime.strptime(strdate1, "%m/%d/%Y %H:%M:%S %p")
            date2 = datetime.datetime.strptime(strdate2, "%m/%d/%Y %H:%M:%S %p")
            dt = date1-date2
            T[i][j] = int(dt.total_seconds() / 60)
    #Apply the function linked to the P and T matrices
    linked(P,T,delta,tau)
    #return the linking distribution
    return shooting_data['spotted'].value_counts()

8) Define  a function to get for a specified year the monthly linkage ratio, total number of shooting incidents, and number of linked shooting incidents

In [9]:
def linkageratio(year,delta,tau,shootings_raw,shotspotter_raw):
    tot= [[]] * 12
    linked = [[]] * 12
    ratio = [[]]*12
    L = [[]]*12
    for mo in np.arange(0,12,1):
        L[mo]=LD(shootings_raw,shotspotter_raw,year,mo+1,delta,tau)
        tot[mo] = np.sum(L[mo])
        if 0 in L[mo].index:
            linked[mo] = tot[mo]-L[mo][0]
        else:
            linked[mo] = tot[mo]
        if tot[mo]>0:
            ratio[mo] = linked[mo]/tot[mo]
        else:
            ratio[mo]=0
    return [ratio,linked,tot]

9) Variables used to store for a specified year the monthly total shootings, total linked shootings, and linking ratios.

In [10]:
yrtot= [[]] * 4
yrlinked=[[]]*4
yrratio=[[]]*4

10) Check linkageratio function for the year 2018

In [11]:
#[yrratio[0],yrlinked[0],yrtot[0]]=linkageratio(2018,delta,tau,shootings_raw,shotspotter_raw) #year 0 =2018

In [12]:
#print(yrratio[0])
#print(yrlinked[0])
#print(yrtot[0])

11) Run the linkageratio function for all 4 years

In [13]:
#for i in np.arange(0,4,1):
#    [yrratio[i],yrlinked[i],yrtot[i]]=linkageratio(2018+i,delta,tau,shootings_raw,shotspotter_raw)

12) Get the total shootings and total number linked between 2018-2021

In [42]:
total_linked = 0
beat_tot = []
beat_tot_ratio = []
beat_tot_linked = []
for k in beat_array:
    beat = k
    shotspotter=shotspotter_raw[shotspotter_raw["BEAT"]==beat]
    shootings=shootings_raw[shootings_raw["BEAT"]==beat]
    shotspotter=shotspotter.reset_index(drop=True)
    shootings=shootings.reset_index(drop=True)
    for i in np.arange(0,4,1):
        [yrratio[i],yrlinked[i],yrtot[i]]=linkageratio(2018+i,delta,tau,shootings,shotspotter)
    linked= [(g + h+i+j)  for g, h,i,j in zip(yrlinked[0],yrlinked[1],yrlinked[2],yrlinked[3])]
    total= [(g + h+i+j) for g, h,i,j in zip(yrtot[0],yrtot[1],yrtot[2],yrtot[3])]
    total_linked = total_linked + sum(linked)
    beat_tot.append(sum(total))
    beat_tot_linked.append(sum(linked))
    beat_tot_ratio.append(100*sum(linked)/sum(total))
    print('Beat ',beat)
    print('2018-21 Total Shooting='+str(sum(total)))
    print('2018-21 Number Linked ='+str(sum(linked)))
    print('2018-21 % Linked ='+str(100*sum(linked)/sum(total)))

Beat  311
2018-21 Total Shooting=51
2018-21 Number Linked =16
2018-21 % Linked =31.372549019607842
Beat  312
2018-21 Total Shooting=111
2018-21 Number Linked =67
2018-21 % Linked =60.36036036036036
Beat  313
2018-21 Total Shooting=80
2018-21 Number Linked =32
2018-21 % Linked =40.0
Beat  314
2018-21 Total Shooting=21
2018-21 Number Linked =8
2018-21 % Linked =38.095238095238095
Beat  321
2018-21 Total Shooting=105
2018-21 Number Linked =47
2018-21 % Linked =44.76190476190476
Beat  322
2018-21 Total Shooting=105
2018-21 Number Linked =57
2018-21 % Linked =54.285714285714285
Beat  323
2018-21 Total Shooting=93
2018-21 Number Linked =48
2018-21 % Linked =51.61290322580645
Beat  324
2018-21 Total Shooting=78
2018-21 Number Linked =47
2018-21 % Linked =60.256410256410255
Beat  331
2018-21 Total Shooting=90
2018-21 Number Linked =35
2018-21 % Linked =38.888888888888886
Beat  332
2018-21 Total Shooting=96
2018-21 Number Linked =56
2018-21 % Linked =58.333333333333336
Beat  333
2018-21 Total S

In [45]:
print(sum(beat_tot))
print(sum(beat_tot_linked))
print(sum(beat_tot_ratio)/12)

915
466
50.31813686360989
